In [ ]:
# This file is part of the prkng_mtl project.
#
# Copyright (C) 2025 coklacour
#
# The prkng_mtl project is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# The prkng_mtl project is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

Solution existante : https://donnees.montreal.ca/showcase/carte-curblr-de-montreal

## Packages

In [ ]:
import os
import pandas as pd
import numpy as np
import fiona
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point

import folium
from folium import plugins

## Macro variables

In [ ]:
INPUT_DIR = os.path.dirname(os.path.abspath(''))+'/datas/raw/'
OUTPUT_DIR= os.path.dirname(os.path.abspath(''))+'/reports/'

## Import

Réseau artériel administratif (RAAV)

In [ ]:
fichier = INPUT_DIR + "/reseau-arteriel-administratif-2023.gpkg"
raav = fiona.listlayers(fichier)
gdf = gpd.read_file(fichier, layer=raav[0])

Park

In [ ]:
park = pd.read_csv(INPUT_DIR+"/park.csv", sep=',')

rpa

In [ ]:
rpa = pd.read_csv(INPUT_DIR+"/rpa.csv", sep=',')

Geobase

In [ ]:
geobase = pd.read_csv(INPUT_DIR+"/geobase.csv", sep=',')

Geobase double

In [ ]:
double = pd.read_csv(INPUT_DIR+"/double.csv", sep=',')

Geobase noeuds

In [ ]:
nodes = pd.read_csv(INPUT_DIR+"/nodes.csv", sep=',')

## Exploration

Focus sur la rue Mentana

### Réseau artériel administratif (RAAV)

In [ ]:
# Mentana
gdf_mentana = gdf[gdf['NOM_VOIE'].str.contains('Mentana')]
gdf_mentana.head(2)

In [ ]:
print("Nombre de lignes concernant la rue Mentana :", gdf_mentana.shape[0])
print("Nombre de troncons sur la rue Mentana :", len(gdf_mentana['ID_TRC'].unique()))

In [ ]:
# convertir le GeoDataFrame vers le système attendu par Folium (EPSG:4326)
gdf_mentana = gdf_mentana.to_crs(epsg=4326)

### Geobase

#### Geobase

In [ ]:
# Mentana
geo_mentana = geobase[geobase['NOM_VOIE'].str.contains('Mentana')]

In [ ]:
# vue d'1 troncon sur la rue Mentana
geo_mentana[geo_mentana['ID_TRC'] == 1220158]

#### Geobase double

In [ ]:
# vue du meme troncon sur la rue Mentana (les 2 trottoirs)
dbl_mentana = double[double['NOM_VOIE'].str.contains('Mentana')]
dbl_mentana[dbl_mentana['ID_TRC'] == 1220158]

#### Geobase noeuds

In [ ]:
nodes.head(2)

Convertir la colonne 'geometry' en objets géométriques shapely

In [ ]:
nodes = nodes.rename(columns={'GEOMETRY': 'geometry'})
nodes['geometry'] = nodes['geometry'].apply(wkt.loads)

In [ ]:
nodes.shape[0]

In [ ]:
# Transformer en GeoDataFrame
nodes = gpd.GeoDataFrame(nodes, geometry='geometry', crs=gdf_mentana.crs)

# Fusionner toutes les géométries de gdf_mentana en une seule (polylignes)
zone_mentana = gdf_mentana.geometry.union_all()


# Option 1 : garder les points qui **touchent ou croisent** les lignes
#nodes_mentana = nodes[nodes.geometry.intersects(zone_mentana)]

# Option 2 (plus souple) : rendre l'intersection plus permissive
zone_mentana_buffered = zone_mentana.buffer(0.0001)
nodes_mentana = nodes[nodes.geometry.intersects(zone_mentana_buffered)]

In [ ]:
nodes_mentana.shape[0]

### Panneaux

In [ ]:
print("Nombre de lignes concernant le stationement :", park.shape[0])
print("Nombre de poteaux :", len(park['POTEAU_ID_POT'].unique()))
print("Nombre de panneaux :", len(park['PANNEAU_ID_PAN'].unique()))

In [ ]:
# exemple d'un poteau avec plusieurs panneaux
park[park['POTEAU_ID_POT'] == 305837]

Identifier les panneaux sur la rue Mentana

In [ ]:
# Créer une colonne geometry à partir des coordonnées
park["geometry"] = park.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
park = gpd.GeoDataFrame(park, geometry="geometry", crs=gdf_mentana.crs)

# Fusionner toutes les géométries de gdf_mentana en une seule (polylignes)
zone_mentana = gdf_mentana.geometry.union_all()

# Option 1 : garder les panneaux qui **touchent ou croisent** les lignes
#park_mentana = park[park.geometry.intersects(zone_mentana)]

# Option 2 (plus souple) : rendre l'intersection plus permissive
zone_mentana_buffered = zone_mentana.buffer(0.0001)
park_mentana = park[park.geometry.intersects(zone_mentana_buffered)]

In [ ]:
park_mentana.shape[0]

## Vizualisation (pour le fun)

Pour chaque ID_TRC, ajouter les numéros civiques réels situé au début / fin à la gauche / droite du tronçon

In [ ]:
gdf_mentana.shape[0]

In [ ]:
cols = ['ID_TRC', 'DEB_GCH', 'FIN_GCH', 'DEB_DRT', 'FIN_DRT']
gdf_mentana['ID_TRC'] = gdf_mentana['ID_TRC'].astype(int)

gdf_mentana = gdf_mentana.merge(
    geo_mentana[cols],
    on='ID_TRC',
    how='left'
)

gdf_mentana.shape[0]

In [ ]:
# Centrer sur le Plateau
m = folium.Map(location=[45.5271, -73.5804],  # Coordonnées du Plateau
               tiles='cartodbpositron',       # Type de carte
               min_zoom=10, max_zoom=20,      # Plage de zoom
               zoom_start=14)                 # Zoom initial pour se concentrer sur le Plateau

# Boucle à travers les données du DataFrame 'gdf_mentana' et ajouter des marqueurs
for idx, row in gdf_mentana.iterrows():
    if row.geometry.geom_type == 'LineString':
        # Extraire les coordonnées de la LineString
        coords = [(pt[1], pt[0]) for pt in row.geometry.coords]  # (lat, lon)

        # Popup personnalisé
        popup_content = f"""
        <b>ID_TRC:</b> {row['ID_TRC']}<br>
        <b>Numéros civiques gauche:</b> ({row['DEB_GCH']} - {row['FIN_GCH']})<br>
        <b>Numéros civiques droite:</b> ({row['DEB_DRT']} - {row['FIN_DRT']})
        """
        popup = folium.Popup(popup_content, max_width=300)

        # Ajouter la ligne à la carte
        folium.PolyLine(locations=coords, color='blue', weight=3, popup=popup).add_to(m)

# Boucle à travers les données du DataFrame 'nodes_mentana' et ajouter des marqueurs
for pt in nodes_mentana.geometry:
    folium.CircleMarker(location=[pt.y, pt.x], radius=3, color='red').add_to(m)
    
# Boucle à travers les données du DataFrame 'park_mentana' et ajouter des marqueurs
for pt in park_mentana.geometry:
    folium.CircleMarker(location=[pt.y, pt.x], radius=3, color='green').add_to(m)

# Afficher la carte
m

In [ ]:
html_map = m._repr_html_()
with open(OUTPUT_DIR+"/mentana.html", "w") as file:
    file.write(html_map)